In [1]:
from utils import *
import pandas as pd
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
lang = "HR.HT"
config = democonfig
config["NUM_EPOCH"] = 30
config["lang"] = lang


config["model_name"] = "classla/bcms-bertic"
config["model_type"] = "bert"
test = (
    pd.read_excel("COPA-MT-MK.BG.TR.MT.IS.HR.xlsx", sheet_name="test")
    .dropna(axis=1, how="all")
    .rename(columns=col_rename_dict)
)
dev = (
    pd.read_excel("COPA-MT-MK.BG.TR.MT.IS.HR.xlsx", sheet_name="dev")
    .dropna(axis=1, how="all")
    .rename(columns=col_rename_dict)
)
train = (
    pd.read_excel("COPA-MT-MK.BG.TR.MT.IS.HR.xlsx", sheet_name="train")
    .dropna(axis=1, how="all")
    .rename(columns=col_rename_dict)
)

for asksfor in reversed("cause effect".split()):
    import torch
    torch.cuda.empty_cache()
    import gc
    gc.collect()
    import pickle
    with open(f"bertic_{asksfor}.pkl", "rb") as f:
        model = pickle.load(f)
    sep_token = model.tokenizer.sep_token
    config["asksfor"] = asksfor
    test_subset = filter_dataframe(
        test,
        lang=lang,
        asksfor=asksfor,
        sep_token=model.tokenizer.sep_token,
        )
    dev_subset = filter_dataframe(
        dev,
        lang=lang,
        asksfor=asksfor,
        sep_token=model.tokenizer.sep_token,
        )
    y_pred_test = eval_model(model, test_subset)
    y_true_test = test.loc[test["asks-for"] == asksfor, "gold"].values.tolist()
    from sklearn.metrics import accuracy_score
    print(f"On test: {accuracy_score(y_true_test, y_pred_test)}")
    results = dict(config)
    results["accuracy"] = accuracy_score(y_true_test, y_pred_test)
    results["split"] = "test"
    results["y_true"] = y_true_test
    results["y_pred"] = y_pred_test
    with open("003_results_new_script.jsonl", "a") as f:
        f.write(str(results)+'\n')

    y_pred_dev = eval_model(model, dev_subset)
    y_true_dev = dev.loc[dev["asks-for"] == asksfor, "gold"].values.tolist()
    print(f"On dev: {accuracy_score(y_true_dev, y_pred_dev)}")
    results = dict(config)
    results["accuracy"] = accuracy_score(y_true_dev, y_pred_dev)
    results["split"] = "dev"
    results["y_true"] = y_true_dev
    results["y_pred"] = y_pred_dev
    with open("003_results_new_script.jsonl", "a") as f:
        f.write(str(results)+'\n')
    del model

  0%|          | 0/250 [00:00<?, ?it/s]

On test: 0.528


  0%|          | 0/48 [00:00<?, ?it/s]

On dev: 0.4375


  0%|          | 0/250 [00:00<?, ?it/s]

On test: 0.54


  0%|          | 0/52 [00:00<?, ?it/s]

On dev: 0.5


In [14]:
import pandas as pd
from ast import literal_eval

with open("003_results_new_script.jsonl") as f:
    contents = f.readlines()
df = pd.DataFrame(data=[literal_eval(i) for i in contents])
gb = df.groupby("split".split()).agg({
    "y_true": sum,
    "y_pred": sum,
})
gb

,y_true,y_pred
split,,
dev,"[2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, ...","[1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, ..."
test,"[1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, ...","[1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, ..."


In [15]:
from sklearn.metrics import accuracy_score

gb["accuracy"] = gb.apply(lambda row: accuracy_score(row["y_true"], row["y_pred"]), axis=1)
gb

,y_true,y_pred,accuracy
split,,,
dev,"[2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, ...","[1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, ...",0.470
test,"[1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, ...","[1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, ...",0.534


In [12]:
df

,lang,NUM_EPOCH,model_name,model_type,asksfor,accuracy,split,y_true,y_pred
0,HR.HT,30,classla/bcms-bertic,bert,effect,0.5280,test,"[1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0, ...","[1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, ..."
1,HR.HT,30,classla/bcms-bertic,bert,effect,0.4375,dev,"[2.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, ...","[1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, ..."
2,HR.HT,30,classla/bcms-bertic,bert,cause,0.5400,test,"[1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, ...","[2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, ..."
3,HR.HT,30,classla/bcms-bertic,bert,cause,0.5000,dev,"[2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, ...","[1, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, ..."
